In [ ]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
file = './data/samples_/level2/sudoku_022'
file_problem = file + '.jpg'
img = cv2.imread(file_problem, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# ぼかし処理
blur = cv2.GaussianBlur(gray, None, 3.0)

In [ ]:
# 大津の二値化
thr, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)

In [ ]:
edge = cv2.Canny(binary, 150, 200)
edge = cv2.dilate(edge, np.ones((11, 11), dtype=edge.dtype))
edge = cv2.erode(edge, np.ones((9, 9), dtype=edge.dtype))

In [ ]:
contours, _ = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

result = img.copy()
cv2.drawContours(result, contours, -1, (255, 0, 0), 3, cv2.LINE_AA)
plt.imshow(result)
plt.axis('off')
plt.show()

In [ ]:
longest_cnt = None
max_length = 0.0
for cnt in contours:
    # 輪郭線の長さを計算
    arclen = cv2.arcLength(cnt, True)
    if max_length < arclen:
        max_length = arclen
        longest_cnt = cnt
arclen = cv2.arcLength(longest_cnt, True)
approx = cv2.approxPolyDP(longest_cnt, arclen * 1.0e-1, True)

In [ ]:
result = img.copy()
cv2.drawContours(result, [approx], -1, (255, 0, 0), 3, cv2.LINE_AA)
plt.imshow(result)
plt.axis('off')
plt.show()

In [ ]:
# 近似輪郭線から頂点を取得 (左上、右上、右下、左下の順番に並べる)
src_pts = approx.reshape((-1, 2)).astype("float32")

# 各頂点が以下の順序で並んでいることを確認します：
# 左上、右上、右下、左下
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    
    # 合計値が最小の点が左上、最大の点が右下
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # 左上
    rect[2] = pts[np.argmax(s)]  # 右下

    # 差分が最小の点が右上、最大の点が左下
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # 右上
    rect[3] = pts[np.argmax(diff)]  # 左下

    return rect

# 頂点を正しい順序に並べ替え
src_pts = order_points(src_pts)

# 縦横比の計算
w = np.linalg.norm(src_pts[1] - src_pts[0])  # 右上 - 左上
h = np.linalg.norm(src_pts[3] - src_pts[0])  # 左下 - 左上
aspect = abs(w) / abs(h)

# 新しい画像サイズを設定
new_w = int(1000 * aspect)
new_h = 1000

# 射影変換先の座標 (左上、右上、右下、左下)
dst_pts = np.array([(0, 0), (new_w, 0), (new_w, new_h), (0, new_h)], dtype="float32")

# 射影変換を計算して、パースをキャンセルする
warp = cv2.getPerspectiveTransform(src_pts, dst_pts)
result = cv2.warpPerspective(img, warp, (new_w, new_h))

In [ ]:
gray_ = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# 大津の二値化
thr_, binary_ = cv2.threshold(gray_, 0, 255, cv2.THRESH_OTSU)

In [ ]:
# 結果画像の高さと幅
h, w = binary_.shape[:2]

# 各セルのサイズを計算
cell_w = w // 9  # 幅を9等分
cell_h = h // 9  # 高さを9等分

In [ ]:
# 各セルを切り出す
cells = []
for i in range(9):
    for j in range(9):
        # 縦横それぞれ上/下/左/右を10%ずつ切り取る
        x1 = j * cell_w + cell_w // 8  # 左端から1/8カット
        y1 = i * cell_h + cell_h // 8  # 上端から1/8カット
        x2 = (j + 1) * cell_w - cell_w // 8  # 右端から1/8カット
        y2 = (i + 1) * cell_h - cell_h // 8  # 下端から1/8カット
        
        # セルを画像から切り出す
        cell = binary_[y1:y2, x1:x2]
        
        # 切り出したセルをリストに追加
        cells.append(cell)

In [ ]:
# 保存したモデルを読み込む
model = joblib.load('model.pkl')
print("モデルが読み込まれました。")

In [ ]:
# 各画像を(64, 64)にリサイズし、1次元配列にフラット化
X = np.array([cv2.resize(cell, (64, 64)).flatten() / 255.0 for cell in cells], dtype='double')

# 予測を行う
predictions = model.predict(X)

# problemリストに予測結果を格納する
problem = []

for prediction in predictions:
    if prediction == 'None':
        problem.append(0)  # Noneの予測結果には0を追加
    else:
        problem.append(int(prediction))  # その他は文字列型をint型に変換して追加

# 結果の表示
print(problem)

In [ ]:
problem = np.array(problem).reshape((9, 9))
print(problem)

In [ ]:
file_answer = file + '.txt'
answer = np.loadtxt(file_answer, dtype = int)
answer

In [ ]:
def check_sudoku_solution(problem, answer):
    
    is_equal = np.array_equal(problem, answer)
    
    if is_equal:
        print("予測と実際の値は一致します。")
    else:
        print("予測と実際の値は一致しません。")
    
    return is_equal


In [ ]:
check_sudoku_solution(problem, answer)

In [ ]:
# ナンプレのボードを表示する関数
def print_board(board):
    for row in board:
        print(" ".join(str(num) if num != 0 else "." for num in row))

In [ ]:
# 数独がルールに従っているかを確認する関数
def is_valid(board, row, col, num):
    # 行に同じ数字があるかを確認
    if num in board[row]:
        return False
    
    # 列に同じ数字があるかを確認
    if num in [board[i][col] for i in range(9)]:
        return False
    
    # 3x3のボックス内に同じ数字があるかを確認
    box_row = row // 3 * 3
    box_col = col // 3 * 3
    for i in range(3):
        for j in range(3):
            if board[box_row + i][box_col + j] == num:
                return False
    
    return True

In [ ]:
# ナンプレをバックトラッキングで解く関数
def solve_sudoku(board):
    for row in range(9):
        for col in range(9):
            # 空いているマス（0）を探す
            if board[row][col] == 0:
                # 1〜9までの数字を試す
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        
                        # 再帰的に解く
                        if solve_sudoku(board):
                            return True
                        
                        # うまくいかない場合は元に戻す
                        board[row][col] = 0
                
                # どの数字も置けない場合はFalseを返す
                return False
    
    return True

In [ ]:
problem

In [ ]:
if solve_sudoku(problem):
    print("\n解けた数独:")
    print_board(problem)
else:
    print("\n解くことができませんでした。")